# Analysis of Kaggle's Heart Failure dataset

Required libraries for data wrangling, statistical analysis and plotting. ~~Plotnine was chosen for similarity to ggplot2 in R.~~ As Plotnine does not have a pairplot API for fast pairwise comparison, I'll switch to Seaborn.

In [ ]:
import pandas as pd
import seaborn as sb

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

## Data loading, inspection and curation
First, load the heart failure dataset supplied by Kaggle and perform some basic introspection on the overall shape of the data and the type of features it contains.

In [ ]:
df = pd.read_csv('heart.csv')
print(df.shape)
print(df.dtypes)
df.head()
#sb.pairplot(df, hue="HeartDisease", corner=True)

This learns us that there are a total of 918 rows and 12 features. Several features have object datatype, but I would prefer category datatype. So for those features I'll check whether the number of distinct objects per feature is small. HeartDisease could be transformed to a boolean, perhaps ExerciseAngina nad FastingBS as well.

In [ ]:
for col in df[["Sex","ChestPainType","FastingBS","RestingECG","ExerciseAngina"]]:
    print(col, df[col].unique())

Above suspicions are confirmed; the respective features have few distinct values each and some can be turned to a boolean. Below the colums are assigned their new data types and a check is done afterwards to assure the conversion was done correctly, such that every feature retains all of its values.

In [ ]:
# Convert to category
#for col in df[["Sex","ChestPainType","RestingECG"]]:
#    df[col] = df[col].astype('category')
# Convert to boolean
#for col in df[["FastingBS","HeartDisease"]]:
#    df[col] = df[col].astype('bool')
df["HeartDisease"] = df["HeartDisease"].astype('bool')
#df["ExerciseAngina"] = df['ExerciseAngina'].replace({'N': 0, 'Y': 1})
#df["ExerciseAngina"] = df["ExerciseAngina"].astype('bool')
print(df.dtypes)
for col in df[["Sex","ChestPainType","FastingBS","RestingECG","ExerciseAngina","HeartDisease"]]:
    print(col, df[col].unique())

## Explorative analysis

There are a variety of interesting feature combinations for an initial explorative analysis, to give some insight on general feature value distributions and correlations.

In [ ]:
# Add binned 5y range age for later
# cut_bins = []
# cut_labels = []
# for i in range(0,20):
#     cut_bins.append(i*5)
#     cut_labels.append(str((i*5)-5)+"-"+str(i*5))
# cut_labels.remove("-5-0") # nr of labels should be bins-1
# df["Bin_Age"] = pd.cut(df["Age"], bins=cut_bins, labels=cut_labels)
# df["Bin_Age"] = df["Bin_Age"].astype("category")

# Distribution of HD per Sex, as counts and percentage of total nr of people
df_dist = (
    df.groupby('Sex', as_index=True)
        .agg(HeartDisease=('HeartDisease', 'sum'),
             Nr_of_people=('HeartDisease', 'count'),
             HD_pct=('HeartDisease', 'mean'))
)
df_dist["HD_pct"] *= 100
print(df_dist)

--

In [ ]:
# Count Male vs Female
sb.countplot(x="Sex",data=df)

# Plot distribution of Male/Female to Age, with median age
g = sb.FacetGrid(df, col="Sex", height=3.5, aspect=1)
g.map(sb.histplot, "Age")
for axes in g.axes.flat:
    _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

--

In [ ]:
# sb.pairplot(df, hue="HeartDisease", corner=True,
# vars = ["Age","RestingBP","Cholesterol","MaxHR","Oldpeak"],
# plot_kws={"alpha":0.4})


In [ ]:
df_m = df.loc[df["Sex"] == "M"]
df_f = df.loc[df["Sex"] == "F"]
for df_x in [df_m,df_f]:
    sb.pairplot(df_x, hue="HeartDisease", corner=True,
    vars = ["Age","RestingBP","Cholesterol","MaxHR","Oldpeak"],
    plot_kws={"alpha":0.4})

### Simple binary classification NN

In [ ]:
# Label encoding
le = LabelEncoder()
for lab in ["Sex","ChestPainType","RestingECG","ExerciseAngina","ST_Slope"]:
    df[lab] = le.fit_transform(df[lab])
print(df.dtypes)

In [ ]:
# Define input/output
X = df.iloc[:, 0:-1] # All features minus HeartDisease
y = df.iloc[:, -1] # HeartDisease

# Define train/test sizes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)

# Standardization input feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Parameters of the model
torch.manual_seed(0)
EPOCHS = 6
BATCH_SIZE = 512
LEARNING_RATE = 0.001

# precision    recall  f1-score   support

#        False       0.87      0.80      0.83       125
#         True       0.87      0.92      0.89       178
# EPOCHS = 11
# BATCH_SIZE = 512
# LEARNING_RATE = 0.01

# recall 0.77 0.90
# EPOCHS = 10
# BATCH_SIZE = 8
# LEARNING_RATE = 0.001

# recall 0.86 0.82
# EPOCHS = 11
# BATCH_SIZE = 12
# LEARNING_RATE = 0.001

# # 0.90
# EPOCHS = 10
# BATCH_SIZE = 8
# LEARNING_RATE = 0.001

# Dataloaders
## train data
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## test data    
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = TestData(torch.FloatTensor(X_test))

# Initialize loaders
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [ ]:
# Define the NN
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        self.layer_1 = nn.Linear(11, 64) # 11 input features in our dataset
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x


In [ ]:
# GPU pls
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Optimizer / loss function
model = BinaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Train the model
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

In [ ]:
# Test model
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
confusion_matrix(y_test, y_pred_list)

In [ ]:
print(classification_report(y_test, y_pred_list))